## Import Packages

In [1]:
import requests
import os
import json
from dateutil import tz
import dateutil.parser as parser
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import pytz

## Grab DFV Tweets

In [2]:
# Grab DFV Unique User ID: 2902349190


bearer_token = os.environ.get("bearer_token")


def create_url():
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
    usernames = "usernames=TheRoaringKitty"
    user_fields = "user.fields=description,created_at"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r


def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    json_response = connect_to_endpoint(url)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()


401


Exception: Request returned an error: 401 {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}

In [ ]:
# Grab DFV Tweet Timeline and Generate CSV

load_dotenv()

bearer_token = os.environ.get("bearer_token")


def create_url():
    user_id = 2902349190
    return "https://api.twitter.com/2/users/{}/tweets".format(user_id)

def create_url_paginated(pagination_token):
    user_id = 2902349190
    return "https://api.twitter.com/2/users/{}/tweets?&pagination_token={}".format(user_id, pagination_token)


def get_params():
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    return {"tweet.fields": "created_at", "max_results" : 100}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserTweetsPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)

    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params)
    
    cols = ['Tweet_Time_EST', 'Tweet_Content']
    tweet_record = [] 
    next_token = json_response['meta']['next_token']

    while(next_token):
        
        data = json_response['data']
        for tweet in data:
            created_at = tweet['created_at']
            datetime_obj = parser.isoparse(created_at)

            my_datetime_est = datetime_obj.astimezone(pytz.timezone('US/Eastern')).strftime('%Y-%m-%d %H:%M:%S')
            
            text = tweet['text']
            tweet_record.append([my_datetime_est, text])
           
    
       
    
        next_url = create_url_paginated(next_token)
        json_response = connect_to_endpoint(next_url, params)
        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
        else:
            data = json_response['data']
            for tweet in data:
                created_at = tweet['created_at']
                datetime_obj = parser.isoparse(created_at)

                my_datetime_est = datetime_obj.astimezone(pytz.timezone('US/Eastern')).strftime('%Y-%m-%d %H:%M:%S')
                
                text = tweet['text']
                tweet_record.append([my_datetime_est, text])
            break

    
    
    dfv = pd.DataFrame(tweet_record, columns=cols)
    dfv.to_csv("./data/output/DFV_Tweets.csv", index=False)
   


if __name__ == "__main__":
    main()


In [ ]:
    df = pd.read_csv("./data/output/DFV_Tweets.csv")
    print(f'Number of Tweets By DFV: {len(df)}')

Number of Tweets By DFV: 1041


## Grab RC Tweets

In [ ]:
# Grab RC Unique User ID: 146058067244486656

bearer_token = os.environ.get("bearer_token")


def create_url():
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
    usernames = "usernames=ryancohen"
    user_fields = "user.fields=description,created_at"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r


def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    json_response = connect_to_endpoint(url)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()


200
{
    "data": [
        {
            "created_at": "2019-07-02T14:08:30.000Z",
            "description": "Founder @Chewy \ud83e\ude91@GameStop",
            "id": "1146058067244486656",
            "name": "Ryan Cohen",
            "username": "ryancohen"
        }
    ]
}


In [ ]:
# Grab RC Tweet Timeline and Generate CSV

load_dotenv()

bearer_token = os.environ.get("bearer_token")


def create_url():
    user_id = 1146058067244486656
    return "https://api.twitter.com/2/users/{}/tweets".format(user_id)

def create_url_paginated(pagination_token):
    user_id = 1146058067244486656
    return "https://api.twitter.com/2/users/{}/tweets?&pagination_token={}".format(user_id, pagination_token)


def get_params():
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    return {"tweet.fields": "created_at", "max_results" : 50}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserTweetsPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)

    # print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params)
    
    cols = ['Tweet_Time_EST', 'Tweet_Content']
    tweet_record = [] 
    next_token = json_response['meta']['next_token']

    while(next_token):
        
        data = json_response['data']
        for tweet in data:
            created_at = tweet['created_at']
            datetime_obj = parser.isoparse(created_at)

            my_datetime_est = datetime_obj.astimezone(pytz.timezone('US/Eastern')).strftime('%Y-%m-%d %H:%M:%S')
            
            text = tweet['text']
            tweet_record.append([my_datetime_est, text])
           
    
        next_url = create_url_paginated(next_token)
        json_response = connect_to_endpoint(next_url, params)
        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
        else:
            data = json_response['data']
            for tweet in data:
                created_at = tweet['created_at']
                datetime_obj = parser.isoparse(created_at)

                my_datetime_est = datetime_obj.astimezone(pytz.timezone('US/Eastern')).strftime('%Y-%m-%d %H:%M:%S')
                
                text = tweet['text']
                tweet_record.append([my_datetime_est, text])
            break

    
    
    rc_df = pd.DataFrame(tweet_record, columns=cols)
    rc_df.to_csv("./data/output/RC_Tweets.csv", index=False)
   


if __name__ == "__main__":
    main()


In [ ]:
    df = pd.read_csv("./data/output/RC_Tweets.csv")
    print(f'Number of Tweets By RC: {len(df)}')   


Number of Tweets By RC: 69
